In [1]:
import numpy as np
import os 
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e24/sample_submission.csv
/kaggle/input/playground-series-s3e24/train.csv
/kaggle/input/playground-series-s3e24/test.csv


In [2]:
import pandas as pd
import plotly.express as px
import lightgbm as lgb
import xgboost as xgb
import optuna
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e24/test.csv")

In [4]:
train

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,159251,40,155,45,69.0,1.5,2.0,1,1,127,...,72,159,14.5,1,0.8,25,26,13,0,0
159252,159252,50,155,75,82.0,1.0,1.0,1,1,120,...,64,108,14.5,1,0.6,21,20,18,0,0
159253,159253,40,160,50,66.0,1.5,1.0,1,1,114,...,87,93,10.9,1,0.6,15,9,12,0,0
159254,159254,50,165,75,92.0,1.2,1.0,1,1,121,...,55,80,14.4,1,1.1,22,17,37,0,1


In [5]:
#train.info()

In [6]:
smoking = train["smoking"]
train = train.drop("smoking", axis=1)

In [7]:
def cleanData(dataset):
    ids = dataset["id"]
    dataset = dataset.drop("id", axis=1)
    
    catData = dataset[['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries']]
    catData = pd.get_dummies(catData, columns = catData.columns)
    
    dataset = dataset.drop(['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries'], axis=1)
    
    
    
    nDataset = (dataset - dataset.mean()) / dataset.std()
    
    nDataset = nDataset.join(catData)
    
    return nDataset, ids

In [8]:
nTrain, trainIds = cleanData(train)
nTest, testIds = cleanData(test)

In [9]:
# def objective_lgb(trial):
#     params = {
#         'n_estimators' : trial.suggest_int('n_estimators',500,1000),
#         "max_depth":trial.suggest_int('max_depth',3,50),
#         "learning_rate" : trial.suggest_float('learning_rate',1e-4, 0.25, log=True),
#         "min_child_weight" : trial.suggest_float('min_child_weight', 0.5,4),
#         "min_child_samples" : trial.suggest_int('min_child_samples',1,250),
#         "subsample" : trial.suggest_float('subsample', 0.2, 1),
#         "subsample_freq" : trial.suggest_int('subsample_freq',0,5),
#         "colsample_bytree" : trial.suggest_float('colsample_bytree',0.2,1),
#         'num_leaves' : trial.suggest_int('num_leaves', 2, 128),
#     }
#     lgbmmodel_optuna = LGBMClassifier(**params,random_state=6)
#     cv = cross_val_score(lgbmmodel_optuna, nTrain, smoking, cv = 4, scoring='roc_auc').mean()
#     return cv

# study = optuna.create_study(direction='maximize')
# study.optimize(objective_lgb, n_trials=100,timeout=2000)

In [10]:
# def objective_xgb(trial):
#     params = {
#         'n_estimators' : trial.suggest_int('n_estimators',500,750),
#         'max_depth':  trial.suggest_int('max_depth',3,50),
#         'min_child_weight': trial.suggest_float('min_child_weight', 2,50),
#         "learning_rate" : trial.suggest_float('learning_rate',1e-4, 0.2,log=True),
#         'subsample': trial.suggest_float('subsample', 0.2, 1),
#         'gamma': trial.suggest_float("gamma", 1e-4, 1.0),
#         "colsample_bytree" : trial.suggest_float('colsample_bytree',0.2,1),
#         "colsample_bylevel" : trial.suggest_float('colsample_bylevel',0.2,1),
#         "colsample_bynode" : trial.suggest_float('colsample_bynode',0.2,1),
#     }
#     xgbmodel_optuna = XGBClassifier(**params,random_state=6,tree_method = "gpu_hist",eval_metric= "auc")
#     cv = cross_val_score(xgbmodel_optuna, nTrain, smoking, cv = 4,scoring='roc_auc').mean()
#     return cv

# study = optuna.create_study(direction='maximize')
# study.optimize(objective_xgb, n_trials=120,timeout=2000)

In [11]:
perams_lgb = {'n_estimators': 835, 'max_depth': 35, 'learning_rate': 0.031055009498070017, 'min_child_weight': 2.1835638592136943, 'min_child_samples': 220, 'subsample': 0.9298705560281547, 'subsample_freq': 5, 'colsample_bytree': 0.34162312179431537, 'num_leaves': 125}
perams_xgb = {'n_estimators': 551, 'max_depth': 13, 'min_child_weight': 49.846543999570784, 'learning_rate': 0.047465437274911496, 'subsample': 0.7827674748408318, 'gamma': 0.8378942824941107, 'colsample_bytree': 0.5997681524662475, 'colsample_bylevel': 0.8678362118325031, 'colsample_bynode': 0.7988100778098284}

In [12]:
model_lgb = lgb.LGBMRegressor(**perams_lgb)
model_xgb = xgb.XGBRegressor(**perams_xgb)

In [13]:
model_xgb.fit(nTrain, smoking)
model_lgb.fit(nTrain, smoking)

LGBMRegressor(colsample_bytree=0.34162312179431537,
              learning_rate=0.031055009498070017, max_depth=35,
              min_child_samples=220, min_child_weight=2.1835638592136943,
              n_estimators=835, num_leaves=125, subsample=0.9298705560281547,
              subsample_freq=5)

In [14]:
# nTest.columns

In [15]:
# nTrain.columns

In [16]:
xgb_pred = model_xgb.predict(nTest)
lgb_pred = model_lgb.predict(nTest)

In [17]:
preds = (xgb_pred*0.5) + (lgb_pred*0.5)

In [18]:
def sigmoid(x):
    return 1 / (1 + np.e**(-x))

In [19]:
preds = sigmoid(preds)

In [20]:
fig = px.scatter(y=preds)
fig.show()

In [21]:
sub = pd.DataFrame()
sub['id'] = testIds
sub['smoking'] = preds
sub.to_csv('submission.csv',index=False)